## import

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import re

from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels

import os

import nibabel as nib
from nilearn.datasets import fetch_surf_fsaverage

import brainstat
from brainstat.stats.terms import FixedEffect

import pandas as pd
from scipy import io
import scipy as sp
import scipy.stats
from scipy import stats

import matplotlib

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
import sys , numpy as np
mod = sys.modules[__name__]

## load subcortex files

In [ ]:
from glob import glob
path = './data/schaefer200/subcortex/'
filepaths = glob(path+'*.csv') # load path of csv files

In [ ]:
import pandas as pd
for i in range(len(filepaths)):
    dat = pd.read_csv(filepaths[i], header=None)
    res = re.split('[\\\/]',filepaths[i])
    globals()[res[-1][:-4]+'_a'] = dat.to_numpy() # load csv files as numpy type

In [ ]:
for i in range(1,50+1):
    globals()['sctx{}'.format(i)] = getattr(mod, 'gp{}_200_a'.format(i)) 
# we set gp{}_200_a as sctx{}

In [ ]:
for i in range(1,50+1):
    globals()['sctx{}'.format(str(i+50))] = getattr(mod, 'gn{}_200_a'.format(i)) 
# we set gn{}_200_a as sctx{}

### make subcortico-subcortical connectivity matrix

In [ ]:
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure 
correlation_measure = ConnectivityMeasure(kind='correlation')

for i in range(1,100+1):
    globals()['sctx{}_c'.format(i)] = correlation_measure.fit_transform([getattr(mod, 'sctx{}'.format(i))]) 

In [ ]:
for i in range(1, 100+1):
    globals()['sctx{}_c'.format(i)] = getattr(mod, 'sctx{}_c'.format(i))[0]
# (1,17,17) array shape -> (17,17) array shape

### 1. z-transformation
### 2. transform all diagonals into zeros

In [ ]:
tab = np.zeros(shape=(17,17))

In [ ]:
for i in range(1,101):
    tab = np.arctanh(getattr(mod, 'sctx{}_c'.format(i)))
    np.fill_diagonal(tab, np.zeros(17))
    globals()['sctx{}_cz'.format(i)] = tab

### get group connectivity matrix averaging z-transformed 100 connectivity matrix

In [ ]:
import numpy
sctx_sum = numpy.zeros(shape=(17,17))
for i in range(1,101):
    sctx_sum = sctx_sum + getattr(mod, 'sctx{}_cz'.format(i))

In [ ]:
sctx_mean = sctx_sum/100

In [ ]:
plotting.plot_matrix(sctx_mean, vmin=0, vmax=1,colorbar=False) # we get group connectivity matrix of 100 individual connectivity matrices

### <span style="color:#2D3748; background-color:#fff5b1;">make subcortical-cortico connectivity matrix</span>

### load cortex files

In [ ]:
from glob import glob
path = './data/schaefer200/cortex/'
filepaths = glob(path+'*.csv') # load path of csv files

In [ ]:
import pandas as pd
for i in range(len(filepaths)):
    dat = pd.read_csv(filepaths[i], header=None)
    res = re.split('[\\\/]',filepaths[i])
    globals()[res[-1][:-4]+'_a'] = dat.to_numpy() # load csv files as array type

In [ ]:
for i in range(1,100+1):
    globals()['ctx{}'.format(i)] = getattr(mod,'g{}_200_a'.format(i))

### make cortico-cortical connectivity matrix

In [ ]:
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure 
correlation_measure = ConnectivityMeasure(kind='correlation')

for i in range(1,100+1):
    globals()['ctx{}_c'.format(i)] = correlation_measure.fit_transform([getattr(mod, 'ctx{}'.format(i))]) 

In [ ]:
for i in range(1, 100+1):
    globals()['ctx{}_c'.format(i)] = getattr(mod, 'ctx{}_c'.format(i))[0]
# (1,200,200) array shape -> (200,200) array shape

### 1. z-transformation
### 2. transform all diagonals into zeros

In [ ]:
tab = np.zeros(shape=(200,200))

In [ ]:
for i in range(1,101):
    tab = np.arctanh(getattr(mod, 'ctx{}_c'.format(i)))
    np.fill_diagonal(tab, np.zeros(200))
    globals()['ctx{}_cz'.format(i)] = tab

### get group connectivity matrix averaging z-transformed 100 connectivity matrix

In [ ]:
import numpy
ctx_sum = numpy.zeros(shape=(200,200))
for i in range(1,101):
    ctx_sum = ctx_sum + getattr(mod, 'ctx{}_cz'.format(i))

In [ ]:
ctx_mean = ctx_sum/100 

### make subcortico-cortical connectivity matrix

In [ ]:
for i in range(1,100+1):
    print('\n\033[1m'+'##{}th person'.format(i))
    npz = np.zeros((200,17))
    x = getattr(mod,'ctx{}'.format(i))
    y = getattr(mod,'sctx{}'.format(i))
    for j in range(0,199+1):
        print('\n\033[1m'+'{}th row'.format(j+1))
        for k in range(0,16+1):
            print('\033[0m' +'{}th col'.format(k+1))
            npz[j][k] = np.corrcoef(x.T[j],y.T[k])[0][1]
            globals()['sc_ctx{}_c'.format(i)] = npz

we make 200x9 matrix averaging left and right regions of 200x17 matrix
> we can get 200x9 matrices of 100 individuals

In [ ]:
for i in range(1,100+1):
    npz = np.zeros((9,200))
    tmp = getattr(mod,'sc_ctx{}_c'.format(i)).T
    for j in range(0,7+1):
        npz[j] = (tmp[j]+tmp[j+8])/2
        npz[8] = tmp[16]
        globals()['sc_ctx{}_c2'.format(i)] = npz

## <span style="color:#2D3748; background-color:#fff5b1;"> make subcortical-weigthed manifold (henceforth eigenvector)</span>

### make template of eigenvectors 1,2 and 3 with hcp

In [ ]:
mat_file_name = "./data/gradients_FC.mat"
mat_file = scipy.io.loadmat(mat_file_name)

In [ ]:
gm_mean = mat_file['gm_mean']

### align group connectivity matrix with hcp eigenvectors

In [ ]:
# Gradient alignment
align = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)

align.fit(ctx_mean, reference = gm_mean)

### align each individual connectivity matrix with hcp-aligned group connectivity matrix

> we call it individual eigenvector

In [ ]:
for i in range(1,100+1):
    align2 = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)
    align2.fit(getattr(mod,'ctx{}_cz'.format(i)), reference = align.aligned_)
    globals()['grad_{}'.format(i)] = align2.aligned_[:,0:2+1]

### we perform element-wise multiplication with subcortico-cortical connectivity matrix and cortical eigenvectors

In [ ]:
for i in range(1,100+1):
    grad = getattr(mod,'grad_{}'.format(i)).T
    sc_ctx = getattr(mod,'sc_ctx{}_c2'.format(i))
    mat = np.zeros((3,9,200))
    for j in range(0,2+1):
        for k in range(0,8+1):
            mat[j,k,:] = np.multiply(grad[j],sc_ctx[k]) 
    globals()['sw_manifold{}'.format(i)] = mat
# array type: 3x9x200(3 eigenvectors x 9 regions x 200 nodes)

## Multivariate analysis

> we perform multivariate analysis with 100 individual eigenvectors

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

brain = pd.read_excel("./data/환자군_정상군_181104_결측값업데이트_scores2_bypark.xlsx")

make sex and grp as dummy variables

In [ ]:
age = brain['Age']
sex = brain['Sex']
sex=np.array(sex)
sex=sex.reshape(100,1)
grp = brain['group']
grp=np.array(grp)
grp=grp.reshape(100,1)
NumSubj = np.size(grp)

In [ ]:
sex_dum1 = np.zeros((NumSubj,1))
sex_dum1 = np.where(sex==0, 1, sex_dum1)
sex_dum2 = np.zeros((NumSubj,1))
sex_dum2 = np.where(sex==1, 1, sex_dum2)
sex_dum = np.concatenate((sex_dum1, sex_dum2), axis=1) # F,M

In [ ]:
grp_dum1 = np.zeros((NumSubj,1))
grp_dum1 = np.where(grp==0, 1, grp_dum1)
grp_dum2 = np.zeros((NumSubj, 1))
grp_dum2 = np.where(grp==1, 1, grp_dum2)
grp_dum = np.concatenate((grp_dum1, grp_dum2), axis=1) # ctl pat

In [ ]:
# get brainstat dir
brainstat_dir = os.path.dirname(brainstat.__file__)

define model

In [ ]:
# from brainstat.stats.terms import FixedEffect

term_age = FixedEffect(age, "age")
term_sex = FixedEffect(sex_dum, {"F", "M"})
term_grp = FixedEffect(grp_dum, {"ctl", "pat"})
model = term_age + term_sex + term_grp

model

### 1. we will make between-group comparison with degree centralities of subcortical-weighted manifolds

- to be specific, we get the values by averaging all nodes of subcortical-weighted manifolds
- Thus, we can get the matrices of 100 individuals, where one matrix array shape is 3x9x1.

### 2. we then indicate them as scatter plots into 3-dimensional spaces -> 2-(a) part

In [ ]:
for i in range(0,8+1):
    sw_mean = np.zeros((100,3))
    for j in range(1,100+1):
        sw_manifold = getattr(mod,'sw_manifold{}'.format(j))
        row = sw_manifold[:,i,:]
        rm = row.mean(axis=1)
        sw_mean[j-1,:] = rm
    globals()['sw_mean{}'.format(str(i+1))] = sw_mean

when we put contrast = patient groups - normal controls, we get t-values of these contrasts

> we perform between-group comparisons in terms of 9 subcortical regions

In [ ]:
from brainstat.stats.SLM import SLM
from  statsmodels.stats import multitest


T_val = []
P_val = []
P_cor = []
for i in range(1,9+1):
    sw_mean = getattr(mod,'sw_mean{}'.format(i))
    Z = np.dstack([sw_mean[0,:],sw_mean[1,:]])
    for j in range(2,99+1):
        Z = np.dstack([Z,sw_mean[j,:]])
    Z = Z[0,:,:].T
    globals()['Z{}'.format(i)] = Z
    
    # t-values
    contrast = grp_dum[:,0] - grp_dum[:,1]
    slm = SLM(model,contrast,correction='fdr')
    slm.fit(Z)
    t = slm.t
    
    # cdf of t-values
    rv = sp.stats.t(df=slm.df)
    cdf = rv.cdf(slm.t)
    
    # p-values of t-values
    p = 1-cdf
    
    # corrected p-values
    pval = stats.t.sf(np.abs(slm.t),slm.df)*2
    pcor = multitest.fdrcorrection(pval.ravel())[1]
    
    T_val.append(t)
    P_val.append(p)
    P_cor.append(pcor)
    
    sig_idx = np.where(pcor<0.055)
    sig_t = np.zeros((200,1))
    sig_t[sig_idx] = t.T[sig_idx]
    globals()['sig_t{}'.format(i)] = sig_t

In [ ]:
region_name = ['Accumbens','Amygdala','Caudate','Hippocampus','Pallidum','Putamen','Thalamus','Cerebellum','Brainstem']

sctx region1~9 t-value

In [ ]:
i=0
print('\033[1m# t_value')
for t in zip(T_val):
    print('\033[0m{}:'.format(region_name[i]),t[0][0][0].round(2))
    i = i+1

sctx region1~9 p-value

In [ ]:
i=0
print('\033[1m# p_value')
for p in zip(P_val):
    print('\033[0m{}:'.format(region_name[i]),p[0][0][0].round(2))
    i = i+1

sctx region 1~9 corrected p_value

In [ ]:
i=0
print('\033[1m# corrected p_value')
for p in P_cor:
    print('\033[0m{}:'.format(region_name[i]),p[0].round(2))
    i = i+1

## Supplement part

### 2-(a) part

make three-dimensional plots of 9 regions and save them as .png-type files

In [ ]:
for i in range(1,9+1):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    pat = np.zeros(3) # patients: red
    ctl = np.zeros(3) # normals: blue
    sw_mean = getattr(mod,'sw_mean{}'.format(i))
    for j in range(0,99+1):
        coord = sw_mean[j,:]
        if j <= 49:
            col = 'red'
            pat = pat + coord
        else: 
            col = 'blue'
            ctl = ctl + coord
        ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
    pat_mean = pat/50
    ctl_mean = ctl/50
    ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
    ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')

#     ax.set_xlim3d(-0.04, 0.02)
#     ax.set_ylim3d(-0.02, 0.02)
#     ax.set_zlim3d(-0.05, 0.03)

#     ax.axes.xaxis.set_ticklabels(['',-0.03,-0.02,-0.01,'0',0.01,''])
#     ax.axes.yaxis.set_ticklabels(['',0.01,'0',0.01,0.02])
#     ax.axes.zaxis.set_ticklabels(['',-0.04,-0.02,'0',0.02,''])

    ax.legend(['pat', 'ctl'])

    plt.title(region_name[i-1])

    ax = plt.gca()
    leg = ax.get_legend()
    
    leg.legendHandles[0].set_color('red')
    leg.legendHandles[1].set_color('blue')
    
#     plt.legend(loc='best')    
#     plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[i-1]))
#     plt.close()

- Accumbens

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(1))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['',-0.02,'','',0.01,''])
ax.axes.yaxis.set_ticklabels(['','',-0.01,'','','',0.01,'',''])
ax.axes.zaxis.set_ticklabels(['',-0.005,'','','',0.015,''])


ax = plt.gca()
leg = ax.get_legend()


# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[1-1]))
#     plt.close()

- Amygdala

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(2))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['',-0.01,'','',0.02,''])
ax.axes.yaxis.set_ticklabels(['',-0.005,'','',0.01,'',''])
ax.axes.zaxis.set_ticklabels(['',-0.005,'','',0.01,''])


ax = plt.gca()
leg = ax.get_legend()



# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[2-1]))
#     plt.close()

- Caudate

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(3))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['','',-0.01,'','','',0.01])
ax.axes.yaxis.set_ticklabels(['','',-0.005,'','',0.01,'',''])
ax.axes.zaxis.set_ticklabels(['',-0.01,'','','',0.01,''])


ax = plt.gca()
leg = ax.get_legend()

# leg.legendHandles[0].set_color('red')
# leg.legendHandles[1].set_color('blue')

#     plt.legend(loc='best')    
# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[3-1]))
#     plt.close()

- Hippocampus

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(4))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['',-0.01,'',0.01])
ax.axes.yaxis.set_ticklabels(['',-0.01,'','','',0.01,'',''])
ax.axes.zaxis.set_ticklabels(['',-0.01,'','','',0.01,''])


ax = plt.gca()
leg = ax.get_legend()



# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[4-1]))
#     plt.close()

- Pallidum

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(5))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['','',-0.03,'','','',0.01,''])
ax.axes.yaxis.set_ticklabels(['','',-0.01,'','','',0.01,'',''])
ax.axes.zaxis.set_ticklabels(['',-0.03,'','','',0.01,''])


ax = plt.gca()
leg = ax.get_legend()

# leg.legendHandles[0].set_color('red')
# leg.legendHandles[1].set_color('blue')

#     plt.legend(loc='best')    
# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[5-1]))
#     plt.close()

- Putamen

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(6))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['','',-0.01,'','','',0.01])
ax.axes.yaxis.set_ticklabels(['','',-0.005,'','',0.01])
ax.axes.zaxis.set_ticklabels(['','',-0.015,'','','','',0.01])


ax = plt.gca()
leg = ax.get_legend()



# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[6-1]))
#     plt.close()

- Thalamus

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(7))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['',-0.02,'','',0.01])
ax.axes.yaxis.set_ticklabels(['',-0.01,'','','',0.01])
ax.axes.zaxis.set_ticklabels(['',-0.01,'','','',0.01])


ax = plt.gca()
leg = ax.get_legend()

# leg.legendHandles[0].set_color('red')
# leg.legendHandles[1].set_color('blue')

#     plt.legend(loc='best')    
# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[7-1]))
#     plt.close()

- Cerebellum

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(8))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['',-0.04,'','','','',0.01])
ax.axes.yaxis.set_ticklabels(['',-0.015,'','','','',0.01])
ax.axes.zaxis.set_ticklabels(['',-0.03,'','','',0.01])


ax = plt.gca()
leg = ax.get_legend()



# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[8-1]))
#     plt.close()

- Brainstem

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
pat = np.zeros(3) 
ctl = np.zeros(3) 
sw_mean = getattr(mod,'sw_mean{}'.format(9))
for j in range(0,99+1):
    coord = sw_mean[j,:]
    if j <= 49:
        col = 'red'
        pat = pat + coord
    else: 
        col = 'blue'
        ctl = ctl + coord
    ax.scatter(coord[0],coord[1],coord[2],c=col,alpha=0.1)
pat_mean = pat/50
ctl_mean = ctl/50
ax.scatter(pat_mean[0],pat_mean[1],pat_mean[2],c='red',marker='D')
ax.scatter(ctl_mean[0],ctl_mean[1],ctl_mean[2],c='blue',marker='D')


ax.axes.xaxis.set_ticklabels(['',-0.01,'','',0.02])
ax.axes.yaxis.set_ticklabels(['','',-0.01,'','','','',0.015])
ax.axes.zaxis.set_ticklabels(['',-0.015,'','','','',0.01])


ax = plt.gca()
leg = ax.get_legend()



# plt.savefig('./fig2/sw_manifold/3d_plot/mean/{}.png'.format(region_name[9-1]))
#     plt.close()

## visualization

### we will plot between-group differenes of subcortical-weighted manifolds onto brain surface using ENIGMA toolbox

In [ ]:
import enigmatoolbox
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical
from enigmatoolbox.plotting.surface_plotting import plot_subcortical

we will load between-group differences of sw-manifolds again

In [ ]:
for i in range(0,8+1):
    sw_mean = np.zeros((100,3))
    for j in range(1,100+1):
        sw_manifold = getattr(mod,'sw_manifold{}'.format(j))
        row = sw_manifold[:,i,:]
        rm = row.mean(axis=1)
        sw_mean[j-1,:] = rm
    globals()['sw_mean{}'.format(str(i+1))] = sw_mean

In [ ]:
T_val = []
P_val = []
P_cor = []
for i in range(1,9+1):
    sw_mean = getattr(mod,'sw_mean{}'.format(i))
    Z = np.dstack([sw_mean[0,:],sw_mean[1,:]])
    for j in range(2,99+1):
        Z = np.dstack([Z,sw_mean[j,:]])
    Z = Z[0,:,:].T
    globals()['Z{}'.format(i)] = Z
    
    # t-values
    contrast = grp_dum[:,0] - grp_dum[:,1]
    slm = SLM(model,contrast,correction='fdr')
    slm.fit(Z)
    t = slm.t
    
    # cdf of t-values
    rv = sp.stats.t(df=slm.df)
    cdf = rv.cdf(slm.t)
    
    # p-values of t-values
    p = 1-cdf
    
    # corrected p-values
    pval = stats.t.sf(np.abs(slm.t),slm.df)*2
    pcor = multitest.fdrcorrection(pval.ravel())[1]
    
    T_val.append(t)
    P_val.append(p)
    P_cor.append(pcor)
    
    sig_idx = np.where(pcor<0.055)
    sig_t = np.zeros((200,1))
    sig_t[sig_idx] = t.T[sig_idx]
    globals()['sig_t{}'.format(i)] = sig_t

In [ ]:
i=0
print('\033[1m# t_value')
for t in zip(T_val):
    print('\033[0m{}:'.format(region_name[i]),t[0][0][0].round(2))
    i = i+1

In [ ]:
tval=[]
for t in zip(T_val):
    tval.append(t[0][0][0].round(2))

sctx region 1~9 pval

In [ ]:
i=0
print('\033[1m# p_value')
for p in zip(P_val):
    print('\033[0m{}:'.format(region_name[i]),p[0][0][0].round(2))
    i = i+1

sctx region 1~9 pcor

In [ ]:
i=0
print('\033[1m# corrected p_value')
for p in P_cor:
    print('\033[0m{}:'.format(region_name[i]),p[0].round(2))
    i = i+1

In [ ]:
pcor=[]
for p in P_cor:
    pcor.append(p[0].round(2))

In [ ]:
dat = pd.DataFrame([tval,pcor])

In [ ]:
dat.columns = region_name

In [ ]:
dat.index = ['t-value','p-value']

we make t-values into array type to plot them into brain surface

In [ ]:
sw_mean_dc = np.array([x[0][0][0].round(2) for x in zip(T_val[:7])])

In [ ]:
sw_mean_dc2 = np.concatenate([sw_mean_dc,sw_mean_dc])

In [ ]:
plot_subcortical(array_name=sw_mean_dc2, ventricles=False, size=(800, 400),
                 cmap='Reds', color_bar=False, color_range=(0,3))

plot only significant regions onto brain surface

In [ ]:
sw_mean_dc = np.array([x[0][0][0].round(2) for x in zip(T_val[:7])])

In [ ]:
sw_mean_dc[0]=0

In [ ]:
sw_mean_dc[3:] = 0

In [ ]:
sw_mean_dc

In [ ]:
sw_mean_dc2 = np.concatenate([sw_mean_dc,sw_mean_dc])

In [ ]:
sw_mean_dc2

In [ ]:
# color map을 ctx와 맞춰줌
plot_subcortical(array_name=sw_mean_dc2, ventricles=False, size=(800, 400),
                 cmap='Reds', color_bar=False)

### radar chart
- if the value is significant, we indicate it with * marker

In [ ]:
from math import pi
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

In [ ]:
spider = pd.DataFrame([x[0][0][0].round(2) for x in zip(T_val)]).T

In [ ]:
spider.columns = region_name

In [ ]:
spider = spider.iloc[:,0:7]

In [ ]:
y = spider.values[0]
y = numpy.concatenate((y,y[0]),axis=None)

In [ ]:
y_sig = np.array((y[1]))

In [ ]:
labels = spider.columns
num_labels = len(labels)
labels = ['','','','','','','']    
    
angles = [x/float(num_labels)*(2*pi) for x in range(num_labels)] ## 각 등분점
angles += angles[:1] ## 시작점으로 다시 돌아와야하므로 시작점 추가
angles_sig = np.array((angles[1]))
    

fig = plt.figure(figsize=(5,5))
# fig.set_facecolor('white')
 
    
ax = plt.subplot(polar=True)
ax.set_theta_offset(pi / 2) ## starting point
ax.set_theta_direction(-1) ## direction
    
plt.xticks(angles[:-1], labels, fontsize=13) ## x-ticks label
ax.tick_params(axis='x', which='major', pad=15) ## make empty spaces between x-axes and ticks
 
ax.set_rlabel_position(0) ## y-axis degree
plt.yticks([-0.5,0,0.5,1,1.5,2,2.5],['-0.5','0','0.5','1','1.5','2','2.5'], fontsize=15) ## y-ticks
plt.ylim(-0.5,2.5)

ax.plot(angles, y, linewidth=2, linestyle='solid', color='black') ## print spider-plot    
ax.plot(angles_sig, y_sig,'go', color='black',marker='*',markersize=20) 

# fig.savefig('./fig2/sw_manifold/spider_plot.png')
plt.show()